In [4]:
import pandas as pd
import re
import io

# === CONFIGURACIÓN ===
archivo_entrada = "septiembre.csv"
archivo_salida = "septiembre_productos_estructurado.csv"

def procesar_csv_con_familias_y_precios(path):
    with io.open(path, 'r', encoding='latin-1', errors='ignore') as f:
        lineas = [ln.strip() for ln in f.readlines() if ln.strip()]

    datos = []
    familia_actual = None
    id_counter = 1

    # Patrón para línea de producto (captura unidades e importe total)
    patron = re.compile(
        r'^([0-9A-Za-zçÇ]{3,10})\s+(.+?)\s+"?([-\d\.,]+)"?\s+"?([-\d\.,]+)"?\s+"?([-\d\.,]+)"?',
        re.UNICODE
    )

    for ln in lineas:
        # Detección de familia (mayúsculas sin dígitos)
        if ln.isupper() and not any(ch.isdigit() for ch in ln):
            familia_actual = ln.strip()
            continue

        m = patron.match(ln)
        if m:
            codigo = m.group(1).strip()
            descripcion = m.group(2).strip()

            # Extraemos unidades e importe total (posición 3 y 4 o 4 y 5 según estructura)
            unidades_str = m.group(3)
            importe_str = m.group(4)

            try:
                unidades = float(unidades_str.replace('.', '').replace(',', '.'))
            except:
                unidades = 0.0

            try:
                importe = float(importe_str.replace('.', '').replace(',', '.'))
            except:
                importe = 0.0

            # Calcular precio por unidad (evitar división por 0)
            precio_unidad = round(importe / unidades, 3) if unidades > 0 else 0.0

            datos.append({
                "ID": id_counter,
                "Código": codigo,
                "Descripción": descripcion,
                "Familia": familia_actual or "SIN FAMILIA",
                "Unidades": unidades,
                "Importe Total (€)": importe,
                "Precio por unidad (€)": precio_unidad
            })
            id_counter += 1

    df = pd.DataFrame(datos)
    return df

# === EJECUCIÓN ===
df_final = procesar_csv_con_familias_y_precios(archivo_entrada)
df_final.to_csv(archivo_salida, index=False, encoding='utf-8-sig')

print("✅ CSV generado correctamente:")
print(df_final.head(10))
print(f"\nGuardado como: {archivo_salida}")



✅ CSV generado correctamente:
   ID  Código       Descripción      Familia  Unidades  Importe Total (€)  \
0   1  001758     Boniato Acomp  SIN FAMILIA     177.0               0.00   
1   2  000001          COCACOLA    REFRESCOS     422.0             677.23   
2   3  000002     FANTA NARANJA    REFRESCOS      63.0             121.77   
3   4  000003         COLA ZERO    REFRESCOS     543.0            1057.50   
4   5  000005          LIMONADA    REFRESCOS       2.0               0.00   
5   6  000010          RED BULL    REFRESCOS       6.0               6.55   
6   7  000011      NESTEA LIMÓN    REFRESCOS      85.0             179.77   
7   8  000013  NESTEA  MARACUYÁ    REFRESCOS      71.0             151.68   
8   9  000014            SPRITE    REFRESCOS       6.0               0.00   
9  10  000015       FANTA LIMÓN    REFRESCOS      86.0             100.41   

   Precio por unidad (€)  
0                  0.000  
1                  1.605  
2                  1.933  
3             